In [22]:
import MeCab
import pandas as pd
import re
from content_clustering.src.vectrizer import ContentVectorizer

# ユーザー辞書を設定
m_t = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -u test/fragrance_user_dic.dic')
#print(m_t)

# メタ付与する文章を読み込み
article_df = pd.read_table('data/perfume_sentence.tsv')

colname = ['surface', 'pos1', 'pos2', 'empty1', 'empty2', 'empty3', 'empty4', 'nomarization', 'category1', 'empty5', 'empty6', 'empty7', 'empty8', 'empty9', 'pos3', 'empty10', 'sign']

In [63]:
feature = []
word_list = []

for i, sentence in enumerate(article_df['description']):
    parsed_s = m_t.parse(sentence).replace('\t',',').split('\n')
    parsed_results = pd.Series(parsed_s).str.split(',').tolist()
    df_raw = pd.DataFrame(parsed_results, columns = colname)
    # 'category1'に「香り」か「印象」属性が登録されている単語をDataFarameに登録
    f_df = df_raw[(df_raw['category1'] == '香り') | (df_raw['category1'] =='印象')]

    # 単語と香りor印象属性を抽出する
    f_m = []
    attr_dic = {}
    for s,c in zip(f_df['surface'],f_df['category1']):
        attr_dic = {'word':s,'category':c}
        word_list.append(s)
        f_m.append(attr_dic)
    feature.append(f_m)

In [78]:
word_class = set(word_list)
meta_parts_pairs = []
target_df = pd.DataFrame(columns = [0,1,2])
for meta_parts1 in word_list:
    for meta_parts2 in word_class:
        if meta_parts1 != meta_parts2:
            meta_parts_pairs.append([meta_parts1,meta_parts2])
            
meta_parts_pairs

[['ローズ', 'レザー'],
 ['ローズ', 'フリージア'],
 ['ローズ', 'フルーティフローラル'],
 ['ローズ', '高貴'],
 ['ローズ', 'マンダリン'],
 ['ローズ', 'ピオニー'],
 ['ローズ', '愛され'],
 ['ローズ', '甘酸っぱい'],
 ['ローズ', 'ハーバル'],
 ['ローズ', 'フローラル系'],
 ['ローズ', 'ヒノキ'],
 ['ローズ', 'シンプル'],
 ['ローズ', 'ビターオレンジ'],
 ['ローズ', '気分転換'],
 ['ローズ', 'ラン'],
 ['ローズ', 'ジューシー'],
 ['ローズ', 'ゆったり'],
 ['ローズ', 'ムース'],
 ['ローズ', '奥深い'],
 ['ローズ', '重厚'],
 ['ローズ', '繊細'],
 ['ローズ', 'リフレッシュ'],
 ['ローズ', '大人かわいい'],
 ['ローズ', 'ほんのり'],
 ['ローズ', '開放感'],
 ['ローズ', '奥ゆかしい'],
 ['ローズ', 'みずみずしい'],
 ['ローズ', '月桂樹'],
 ['ローズ', 'クール'],
 ['ローズ', '凛とした'],
 ['ローズ', 'スパイス'],
 ['ローズ', '大人っぽい'],
 ['ローズ', 'ミステリアス'],
 ['ローズ', 'シャンプー'],
 ['ローズ', 'ジャスミン'],
 ['ローズ', 'オレンジ'],
 ['ローズ', '落ち着く'],
 ['ローズ', '清らか'],
 ['ローズ', '柑橘類'],
 ['ローズ', 'アクセント'],
 ['ローズ', 'パチュリ'],
 ['ローズ', '爽やか'],
 ['ローズ', 'シダー'],
 ['ローズ', 'ディープ'],
 ['ローズ', '儚げな'],
 ['ローズ', 'オリエンタル'],
 ['ローズ', 'キリッとした'],
 ['ローズ', 'セクシー'],
 ['ローズ', 'ジンジャー'],
 ['ローズ', 'かっこいい'],
 ['ローズ', '洋梨'],
 ['ローズ', '明るい'],
 ['ローズ', '水'],
 ['ローズ', 'ココナッツ'],
 ['ローズ', 'ムスク'],
 ['

In [20]:
import networkx as nx
from networkx.drawing.nx_agraph import write_dot
import matplotlib.pyplot as plt

def network(
    values:list, 
    node_config:dict,
    node_v = 0.1,
    edge_v = 0.002,
):

    plt.figure(figsize=(15, 15))

    #新規グラフを作成
    #G = nx.MultiDiGraph()
    G = nx.DiGraph()

    G.add_weighted_edges_from(values)
    node_size = []    
    color_map = []
    for node in G:
        if str(node).startswith("to_ad_"):
            color_map.append('red')
        elif str(node).startswith("to_article_"):
            color_map.append('yellow')
        elif str(node).startswith("to_"):
            color_map.append('red')            
        else: 
            color_map.append('green')
            

        size = to_min(node_config.get(str(node), 10))
        
        node_size.append(math.log(size, 2) * node_v)
           
    pos = nx.spring_layout(G, k=100)
    #pos = nx.spiral_layout(G)    
    #top = nx.bipartite.sets(G)[0]
    #pos = nx.bipartite_layout(G, top)    
    
    #pos = nx.shell_layout(G)
    bbox = dict(color='white', alpha=0.5, edgecolor=None)
    edge_labels = {(i, j): w['weight'] for i, j, w in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, alpha=0.5, bbox=bbox, edge_labels=edge_labels)
    
    nx.draw(
        G, 
        pos, 
        node_color=color_map, 
        with_labels=True, 
        alpha=0.7,
        width=[to_min(w['weight'] * edge_v) for i, j, w in G.edges(data=True)],
        node_size=node_size, 
        font_family='IPAexGothic'
    )
    
    #write_dot(G,'graph.dot')
    
    #nx.draw_networkx(G, pos, node_color=color_map, with_labels=True, node_size=node_size, alpha=0.5, font_family='IPAexGothic')

    # 表示
    plt.axis("off")
    plt.show()

In [ ]:
target_df = pd.DataFrame(targets['all'].values()).sort_values(by=[2], ascending=False)
target_df.head(20)

network(target_df.values.tolist(), target_agg, 200, 0.005)